# REINFORCE

The following code contains an implementation of the REINFORCE algorithm, **without Off Policy Correction, LSTM state encoder, and Noise Contrastive Estimation**. Look for these in other notebooks.

Also, I am not google staff, and unlike the paper authors, I cannot have online feedback concerning the recommendations.

**I use actor-critic for reward assigning.** In a real-world scenario that would be done through interactive user feedback, but here I use a neural network (critic) that aims to emulate it.

### **Note on this tutorials:**
**They mostly contain low level implementations explaining what is going on inside the library.**

**Most of the stuff explained here is already available out of the box for your usage.**

In [1]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch.distributions import Categorical

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


# == recnn ==
import sys
sys.path.append("../../")
import recnn

cuda = torch.device('cuda')

# ---
frame_size = 10
batch_size = 10
n_epochs   = 100
plot_every = 30
step       = 0
num_items    = 5000 # n items to recommend. Can be adjusted for your vram 
# --- 
tqdm.pandas()


from jupyterthemes import jtplot
jtplot.style(theme='grade3')

## I will drop low freq items because it doesnt fit into my videocard vram

In [2]:

def prepare_dataset(df, key_to_id, frame_size, env, sort_users=False, **kwargs):
    
    global num_items
    
    value_counts = df['movieId'].value_counts() 
    print('counted!')
    
    # here n items to keep are adjusted
    num_items = 5000
    to_remove = df['movieId'].value_counts().sort_values()[:-num_items].index
    to_keep = df['movieId'].value_counts().sort_values()[-num_items:].index
    to_remove_indices = df[df['movieId'].isin(to_remove)].index
    num_removed = len(to_remove)
    
    df.drop(to_remove_indices, inplace=True)
    print('dropped!')
    
    print('before', env.embeddings.size(), len(env.movie_embeddings_key_dict))
    for i in list(env.movie_embeddings_key_dict.keys()):
        if i not in to_keep:
            del env.movie_embeddings_key_dict[i]
        
    env.embeddings, env.key_to_id, env.id_to_key = recnn.data.utils.make_items_tensor(env.movie_embeddings_key_dict)
    
    print('after', env.embeddings.size(), len(env.movie_embeddings_key_dict))
    print('embeddings automatically updated')
    print('action space is reduced to {} - {} = {}'.format(num_items + num_removed, num_removed,
                                                           num_items))
    
    return recnn.data.prepare_dataset(df=df, key_to_id=env.key_to_id, env=env,
                                      frame_size=frame_size, sort_users=sort_users)


In [3]:
def batch_contstate_discaction(batch, item_embeddings_tensor, frame_size, num_items, *args, **kwargs):
    
    """
    Embed Batch: continuous state discrete action
    """
    
    from recnn.data.utils import get_irsu
    
    items_t, ratings_t, sizes_t, users_t = get_irsu(batch)
    items_emb = item_embeddings_tensor[items_t.long()]
    b_size = ratings_t.size(0)

    items = items_emb[:, :-1, :].view(b_size, -1)
    next_items = items_emb[:, 1:, :].view(b_size, -1)
    ratings = ratings_t[:, :-1]
    next_ratings = ratings_t[:, 1:]

    state = torch.cat([items, ratings], 1)
    next_state = torch.cat([next_items, next_ratings], 1)
    action = items_t[:, -1]
    reward = ratings_t[:, -1]

    done = torch.zeros(b_size)
    done[torch.cumsum(sizes_t - frame_size, dim=0) - 1] = 1
    
    one_hot_action = torch.zeros(action.size(0), num_items)
    one_hot_action.scatter_(1, action.view(-1,1), 1)

    batch = {'state': state, 'action': one_hot_action, 'reward': reward, 'next_state': next_state, 'done': done,
             'meta': {'users': users_t, 'sizes': sizes_t}}
    return batch

def embed_batch(batch, item_embeddings_tensor, *args, **kwargs):
    return batch_contstate_discaction(batch, item_embeddings_tensor, frame_size=frame_size, num_items=num_items)

In [4]:
# embeddgings: https://drive.google.com/open?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
env = recnn.data.env.FrameEnv('../../data/embeddings/ml20_pca128.pkl',
                              '../../data/ml-20m/ratings.csv', frame_size, batch_size,
                              embed_batch=embed_batch, prepare_dataset=prepare_dataset,
                              num_workers = 0)

counted!
dropped!
before torch.Size([27278, 128]) 27278
after torch.Size([5000, 128]) 5000
embeddings automatically updated
action space is reduced to 26744 - 21744 = 5000


In [5]:
class DiscreteActor(nn.Module):
    def __init__(self, hidden_size, num_inputs, num_actions):
        super(DiscreteActor, self).__init__()

        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_actions)
        
        self.saved_log_probs = []
        self.rewards = []

    def forward(self, inputs):
        x = inputs
        x = F.relu(self.linear1(x))
        action_scores = self.linear2(x)
        return F.softmax(action_scores)
    
    
    def select_action(self, state):
        probs = self.forward(state)
        m = Categorical(probs)
        action = m.sample()
        self.saved_log_probs.append(m.log_prob(action))
        return action, probs

### Because I do not have a dynamic environment, I also will include a critic. If you have a real non static environment, you can do w/o citic.

In [6]:
class ChooseREINFORCE():
    
    def __init__(self, method=None):
        if method is None:
            method = ChooseREINFORCE.reinforce
        self.method = method
    
    @staticmethod
    def basic_reinforce(policy, returns, *args, **kwargs):
        policy_loss = []
        for log_prob, R in zip(policy.saved_log_probs, returns):
            policy_loss.append(-log_prob * R)
        policy_loss = torch.cat(policy_loss).sum()
        return policy_loss
    
    @staticmethod
    def reinforce_with_correction():
        raise NotImplemented

    def __call__(self, policy, optimizer, learn=True):
        R = 0
        
        returns = []
        for r in policy.rewards[::-1]:
            R = r + 0.99 * R
            returns.insert(0, R)
            
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 0.0001)

        policy_loss = self.method(policy, returns)
        
        if learn:
            optimizer.zero_grad()
            policy_loss.backward()
            optimizer.step()
        
        del policy.rewards[:]
        del policy.saved_log_probs[:]

        return policy_loss

In [7]:
# === reinforce settings ===

params = {
    'reinforce': ChooseREINFORCE(ChooseREINFORCE.basic_reinforce),
    'gamma'      : 0.99,
    'min_value'  : -10,
    'max_value'  : 10,
    'policy_step': 10,
    'soft_tau'   : 0.001,
    
    'policy_lr'  : 1e-5,
    'value_lr'   : 1e-5,
    'actor_weight_init': 54e-2,
    'critic_weight_init': 6e-1,
}

# === end ===

In [8]:
nets = {
    'value_net': recnn.nn.Critic(1290, num_items, 2048, params['critic_weight_init']).to(cuda),
    'target_value_net': recnn.nn.Critic(1290, num_items, 2048, params['actor_weight_init']).to(cuda).eval(),
    
    'policy_net':  DiscreteActor(2048, 1290, num_items).to(cuda),
    'target_policy_net': DiscreteActor(2048, 1290, num_items).to(cuda).eval(),
}


# from good to bad: Ranger Radam Adam RMSprop
optimizer = {
    'value_optimizer': recnn.optim.Ranger(nets['value_net'].parameters(),
                                          lr=params['value_lr'], weight_decay=1e-2),

    'policy_optimizer': recnn.optim.Ranger(nets['policy_net'].parameters(),
                                           lr=params['policy_lr'], weight_decay=1e-5)
}


loss = {
    'test': {'value': [], 'policy': [], 'step': []},
    'train': {'value': [], 'policy': [], 'step': []}
    }

debug = {}

writer = SummaryWriter(log_dir='../../runs')
plotter = recnn.utils.Plotter(loss, [['value', 'policy']],)
device = cuda

In [9]:
def reinforce_update(batch, params, nets, optimizer,
                     device=torch.device('cpu'),
                     debug=None, writer=recnn.utils.DummyWriter(),
                     learn=False, step=-1):
    
    state, action, reward, next_state, done = recnn.data.get_base_batch(batch)
    
    predicted_action, predicted_probs = nets['policy_net'].select_action(state)
    reward = nets['value_net'](state, predicted_probs).detach()
    nets['policy_net'].rewards.append(reward.mean())
    
    value_loss = recnn.nn.value_update(batch, params, nets, optimizer,
                     writer=writer,
                     device=device,
                     debug=debug, learn=learn, step=step)
    
    
    
    if step % params['policy_step'] == 0 and step > 0:
        
        policy_loss = params['reinforce'](nets['policy_net'], optimizer['policy_optimizer'], learn=learn)
        del nets['policy_net'].rewards[:]
        del nets['policy_net'].saved_log_probs[:]
        print('step: ', step, '| value:', value_loss.item(), '| policy', policy_loss.item())
    
        recnn.utils.soft_update(nets['value_net'], nets['target_value_net'], soft_tau=params['soft_tau'])
        recnn.utils.soft_update(nets['policy_net'], nets['target_policy_net'], soft_tau=params['soft_tau'])

        losses = {'value': value_loss.item(),
                  'policy': policy_loss.item(),
                  'step': step}

        recnn.utils.write_losses(writer, losses, kind='train' if learn else 'test')

        return losses

In [10]:
step = 0
for epoch in range(n_epochs):
    for batch in tqdm(env.train_dataloader):
        loss = reinforce_update(batch, params, nets, optimizer,
                     writer=writer,
                     device=device,
                     debug=debug, learn=True, step=step)
        if loss:
            plotter.log_losses(loss)
        step += 1
        if step % plot_every == 0:
            # clear_output(True)
            print('step', step)
            #test_loss = run_tests()
            #plotter.log_losses(test_loss, test=True)
            #plotter.plot_loss()
        #if step > 1000:
        #    pass
        #    assert False

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


step:  10 | value: 24.99452781677246 | policy -2379.3642578125
step:  20 | value: 25.838939666748047 | policy -7025.1083984375
step 30
step:  30 | value: 21.31504249572754 | policy -6453.48779296875
step:  40 | value: 24.557893753051758 | policy -17982.73046875
step:  50 | value: 24.8399715423584 | policy -2648.5126953125
step 60
step:  60 | value: 22.109338760375977 | policy -1317.727783203125
step:  70 | value: 22.422393798828125 | policy 3406.505859375
step:  80 | value: 23.30132484436035 | policy -23820.28515625
step 90
step:  90 | value: 23.076168060302734 | policy 9186.025390625
step:  100 | value: 22.222537994384766 | policy -3152.271484375
step:  110 | value: 22.635631561279297 | policy 18650.52734375
step 120
step:  120 | value: 22.85053062438965 | policy 67.15323638916016
step:  130 | value: 23.42321014404297 | policy 15718.7001953125
step:  140 | value: 19.7723445892334 | policy 22986.236328125
step 150
step:  150 | value: 17.86973762512207 | policy 11883.517578125
step:  16

KeyboardInterrupt: 